In [1]:
import os
import pandas as pd
import numpy as np

In [111]:
def percent_by_polling_district(riding, year):
    print "Riding: " + str(riding) + ", Year: " + str(year)
    fileName = "pollresults_resultatsbureau" + str(riding) + ".csv"
    filePath = os.path.join(folderName, fileName)
    # Load the data
    pollData = pd.read_csv(filePath)
    
    # Get column names, and remove French portions
    colNames = list(pollData.columns.values)
    colNames = [x.split('/')[0] for x in colNames]
    pollData.columns = colNames
    
    # Drop unnecessary columns
    listColDrop = ['Electoral District Name_English',
                   'Electoral District Name_French',
                   'Void Poll Indicator',
                   'No Poll Held Indicator', 
                   'Merge With',
                   'Rejected Ballots for Polling Station',
                   'Political Affiliation Name_French',
                   "Candidate's First Name",
                   "Candidate's Family Name",
                   "Candidate's Middle Name", 
                   'Incumbent Indicator',
                   'Elected Candidate Indicator']
    pollData = pollData.drop(listColDrop, axis=1)
    
    # Strip the polling ID column of whitespace.
    polCol = 'Polling Station Number'
    s = lambda x: str(x).strip(" ")
    pollData[polCol] = pollData[polCol].map(s)
    
    # Create a pivot table of the data by polling district/candidate name
    pollData = pollData.pivot(
        index='Polling Station Number',
        columns='Political Affiliation Name_English',
        values='Candidate Poll Votes Count')
    # Turn the index back into a column
    pollData.reset_index(level=0, inplace=True)
    
    # Strip the letters off polling stations since the geospatial data
    #  does not include these letters.
    stripCharacters = "ABCDEFG"
    s = lambda x: str(x).strip(stripCharacters)
    statCol = 'Polling Station Number'
    pollData[statCol] = pollData[statCol].map(s)
    
    # Merge polling stations
    pollData = pollData.groupby('Polling Station Number').sum()
    pollData.reset_index(level=0, inplace=True)
    
    # Get the vote totals
    pollData['Vote Totals'] = pollData.sum(axis=1, numeric_only=True)

    # Calculate the percent for each

    # Grab the data we want converted to a percent
    numColsPollData = len(pollData.columns)
    pollDataPercent = pollData.iloc[:, range(1, numColsPollData-1)].copy()

    # Divide it by the total votes for each polling district
    pollDataPercent = pollDataPercent.div(pollData['Vote Totals'], axis=0)

    pollDataPercent = np.round(pollDataPercent*100, decimals=2)

    # Rename columns
    colNames = list(pollDataPercent.columns.values)
    colNames = [x + " (%)" for x in colNames]
    pollDataPercent.columns = colNames

    # Merge it with the original data set
    pollData = pd.concat([pollData, pollDataPercent], axis=1)
    
    # Add back column for electoral district
    pollData['District'] = riding
    
    return pollData

In [112]:
# Create an array for all the years we're interested in
years = [2011]

In [113]:
for year in years:
    # Get list of riding numbers
    folderName = "pollresults_resultatsbureau_canada" + str(year)
    # This file contains the provinces and their electoral districts
    ridingFile = os.path.join(folderName, "RidingListClean.csv")
    ridingList = pd.read_csv(ridingFile)
    
    # We only need the first three columns
    ridingList = ridingList.iloc[:, [0, 2]]
    ridingList = ridingList[ridingList["Province2"] == "New Brunswick"]
    ridings = ridingList.ix[:, 1]
    
    # Is this the first time looping for this year?
    first = True
    for riding in ridings:
        pollData = percent_by_polling_district(riding, year)
        if first == True:
            combined = pollData
        else:
            combined = pd.concat([combined, pollData], axis=0, ignore_index=True)
        first = False

    # Reorder columns before writing
    cols = list(combined)
    cols.insert(0, cols.pop(cols.index('Polling Station Number')))
    cols.insert(0, cols.pop(cols.index('District')))
    combined = combined.ix[:, cols]
    
    fileName = str(year) + "Combined.csv"
    filePath = os.path.join("Output", fileName)
    combined.to_csv(filePath, index=False, encoding='utf-8')

Riding: 13001, Year: 2011
Riding: 13002, Year: 2011
Riding: 13003, Year: 2011
Riding: 13004, Year: 2011
Riding: 13005, Year: 2011
Riding: 13006, Year: 2011
Riding: 13007, Year: 2011
Riding: 13008, Year: 2011
Riding: 13009, Year: 2011
Riding: 13010, Year: 2011
